# Calibration Status to Alarm Automation

This notebook monitors asset calibration status and automatically creates or updates alarms in SystemLink based on the calibration state of managed assets.

## Severity Mapping

The notebook maps asset calibration statuses to alarm severity levels:

- **PAST_RECOMMENDED_DUE_DATE** → Severity 2 (Moderate)  
  Asset is past its recommended calibration due date - creates or updates alarm "Calibration Overdue" with moderate severity

- **APPROACHING_RECOMMENDED_DUE_DATE** → Severity 1 (Low)  
  Asset is approaching its recommended calibration due date - creates or updates alarm "Calibration Due" with low severity

- **OK** → Severity -1 (Clear)  
  Asset calibration is up to date - clears any existing alarms

## Imports and Setup
This section imports required libraries

In [ ]:
import importlib.util

if importlib.util.find_spec("nisystemlink.clients.notification") is None:
    %pip install --upgrade nisystemlink-clients

In [ ]:
from datetime import datetime, timezone
from typing import Any, Dict, List
from urllib.parse import urlsplit, urlunsplit
from typing import Callable, Iterator
import scrapbook as sb
from nisystemlink.clients.alarm import AlarmClient
from nisystemlink.clients.alarm.models import (
    Alarm,
    ClearAlarmTransition,
    CreateOrUpdateAlarmRequest,
    QueryAlarmsWithFilterRequest,
    SetAlarmTransition,
)
from nisystemlink.clients.assetmanagement import AssetManagementClient
from nisystemlink.clients.assetmanagement.models import (
    Asset,
    CalibrationStatus,
    QueryAssetsRequest,
)
from nisystemlink.clients.notification import NotificationClient
from nisystemlink.clients.notification.models import (
    DynamicNotificationConfiguration,
    DynamicNotificationStrategy,
    DynamicStrategyRequest,
    SmtpAddressFields,
    SmtpAddressGroup,
    SmtpMessageTemplate,
    SmtpMessageTemplateFields,
)
from nisystemlink.clients.systems import SystemsClient
from nisystemlink.clients.systems.models import QuerySystemsRequest

## Configuration and Parameters

In this section, we define key parameters and configuration values that will be used throughout the notebook.

**Configuration:**
- **`workspace_ids`**: List of workspace IDs to monitor (empty list = all workspaces)
  - Example: `["workspace-123", "workspace-456"]` or `[]` for all workspaces
- **`calibration_subscribers`**: List of email addresses to receive notifications about calibration status changes
  - Example: `["user@example.com", "admin@example.com"]`
- **`http_url`**: Base SystemLink URL for building asset links in email notifications
- **`CALIBRATION_STATUS_TO_SEVERITY`**: Mapping of calibration status to alarm severity levels (1=Low, 2=Moderate, -1=Clear)
- **`CALIBRATION_STATUS_TO_LABEL`**: Mapping of calibration status to human-readable alarm labels

In [ ]:
workspace_ids: List[str] = []
calibration_subscribers: List[str] = []

# Base SystemLink URL for building asset links in email notifications
http_url = "https://abc.example.com/"

# Calibration status -> desired severity mapping
CALIBRATION_STATUS_TO_SEVERITY = {
    CalibrationStatus.PAST_RECOMMENDED_DUE_DATE.value: 2,
    CalibrationStatus.APPROACHING_RECOMMENDED_DUE_DATE.value: 1,
    CalibrationStatus.OK.value: -1,
}

CALIBRATION_STATUS_TO_LABEL = {
    CalibrationStatus.PAST_RECOMMENDED_DUE_DATE.value: "Calibration Overdue",
    CalibrationStatus.APPROACHING_RECOMMENDED_DUE_DATE.value: "Calibration Due",
    CalibrationStatus.OK.value: "Calibration up to date",
}

## SLE API Utilities

In [ ]:
asset_client = AssetManagementClient()
system_client = SystemsClient()
alarm_client = AlarmClient()
notification_client = NotificationClient()

In [ ]:
def __batch_query(
    query_request: QueryAssetsRequest | QuerySystemsRequest | QueryAlarmsWithFilterRequest,
    query_func: Callable[
        [QueryAssetsRequest | QuerySystemsRequest | QueryAlarmsWithFilterRequest], Asset | Dict[str, Any] | Alarm,
    ],
    paged_data_field: str,
) -> Iterator[List[Any]]:
    """
    Execute a paginated query and yield results in batches.

    Args:
        query_request:
            Request object used for querying. May support pagination via`continuation_token`, `skip`.
        query_func:
            Callable that executes the query using `query_request` and
        paged_data_field:
            Name of the attribute on the response object that contains the list of returned items.

    Yields:
        List[Any]:
            A list of items returned for each page of results.
    """

    while True:
        response = query_func(query_request)
        items = getattr(response, paged_data_field, None)

        if not items:
            break

        yield items

        # Continuation-token based pagination (Alarms)
        continuation_token = getattr(response, "continuation_token", None)
        if continuation_token:
            query_request.continuation_token = continuation_token
            continue

        # Skip based pagination (Systems / Assets)
        if hasattr(query_request, "skip"):
            prev_skip = query_request.skip
            query_request.skip = (query_request.skip or 0) + len(items)

            if query_request.skip == prev_skip:
                break
            continue

        # No pagination mechanism
        break

In [ ]:
def resolve_asset_location(asset: Asset) -> tuple[str | None, str | None, str | None]:
    """
    Resolve the location information for an asset.

    Prioritizes system location (via minion_id) over physical location.

    Args:
        asset (Asset): Asset object containing location information

    Returns:
        tuple[str | None, str | None, str | None]: A tuple containing:
            - resolved_location: Human-readable location (system alias or physical location)
            - system_id: System ID if asset is in a system, None otherwise
            - system_alias: System alias if asset is in a system, None otherwise
    """
    location = asset.location
    if not location:
        return None, None, None

    query_system_request = QuerySystemsRequest(
        skip=0,
        take=100,
        filter=f'id == "{location.minion_id}"',
        projection="new(id,alias)",
    )

    systems_iter: Iterator[List[Dict[str, Any]]] = __batch_query(
        query_request=query_system_request,
        query_func=system_client.query_systems,
        paged_data_field="data",
    )

    try:
        systems = next(systems_iter)
        system = systems[0]
    except (StopIteration, IndexError):
        return None, None, None

    system_id = system.get("id")
    system_alias = system.get("alias")

    resolved_location = system_alias or getattr(location, "physical_location", None)
    return resolved_location, system_id, system_alias

In [ ]:
def get_asset_path(asset: Asset) -> str:
    """
    Generate a unique asset path for alarm identification.

    Constructs a hierarchical path using vendor, model, and serial number.

    Args:
        asset (Asset): Asset object containing vendor, model, and serial information

    Returns:
        str: Formatted asset path string
    """
    resolved_asset_location = resolve_asset_location(asset=asset)
    location = resolved_asset_location[0]
    vendor = asset.vendor_name or str(asset.vendor_number or "")
    model = asset.model_name or str(asset.model_number or "")
    serial_number = str(asset.serial_number or "")
    if location:
        return f"Assets.{location}.{vendor}.{model}.{serial_number}.Calibration"

    return f"Assets.{vendor}.{model}.{serial_number}.Calibration"

In [ ]:
def send_email_notification(
    to_addresses: List[str] | None = None,
    subject: str = "subject",
    body: str = "body",
) -> None:
    """
    Send email notifications using SystemLink's notification service.

    Args:
        to_addresses (List[str] | None): List of recipient email addresses
        subject (str): Email subject line
        body (str): Email body content

    Returns:
        None
    """
    if not to_addresses:
        print("No email recipients configured - skipping email notification")
        return

    # Build SMTP address group with recipient addresses
    address_group = SmtpAddressGroup(fields=SmtpAddressFields(toAddresses=to_addresses))

    # Build SMTP message template with subject and body
    message_template = SmtpMessageTemplate(
        fields=SmtpMessageTemplateFields(subject_template=subject, body_template=body)
    )

    # Create notification configuration
    notification_config = DynamicNotificationConfiguration(
        address_group=address_group, message_template=message_template
    )

    # Create notification strategy
    notification_strategy = DynamicNotificationStrategy(
        notification_configurations=[notification_config]
    )

    # Create request and send notification
    request = DynamicStrategyRequest(notification_strategy=notification_strategy)
    notification_client.apply_dynamic_notification_strategy(request)
    print("Email notification sent successfully")

## Alarm helpers

In [ ]:
def build_calibration_description(
    alarm_display_name: str,
    due_verb_phrase: str,
    resolved_location: str | None,
    asset: Asset,
) -> str:
    """
    Build a calibration alarm description with optional date and location information.

    Args:
        alarm_display_name (str): The asset display name for the alarm
        due_verb_phrase (str): The verb phrase (e.g., "has exceeded", "is approaching")
        resolved_location (str | None): The resolved location (system alias or physical location, or None)
        asset (Asset): The asset object containing calibration information

    Returns:
        str: Formatted description string
    """
    date_phrase = ""
    if asset.external_calibration and asset.external_calibration.resolved_due_date:
        try:
            calibration_date = asset.external_calibration.resolved_due_date
            calibration_date_str = calibration_date.strftime("%B %d, %Y")
            date_phrase = f" of {calibration_date_str}"
        except Exception:
            pass  # Leave date_phrase empty if formatting fails

    description = f"The {alarm_display_name} {due_verb_phrase} the next recommended calibration date{date_phrase}."

    # Only add location sentence if we have a location
    if resolved_location:
        description += (
            f" You can find this device in the following location: {resolved_location}."
        )

    return description


def build_transition_payload(
    asset: Asset,
    alarm_id: str,
    target_severity: int,
    calibration_status: str,
) -> CreateOrUpdateAlarmRequest:
    """
    Build alarm transition payload for creating or updating an alarm.

    Uses template strings for display_name and description with placeholders that are
    dynamically resolved using properties from the transition.

    Args:
        asset (Asset): Asset object containing asset details
        alarm_id (str): Unique identifier for the alarm
        target_severity (int): Desired severity level (-1 for clear, 1 for low, 2 for moderate)
        calibration_status (str): Current calibration status string

    Returns:
        CreateOrUpdateAlarmRequest: Request object for creating or updating an alarm
    """
    display_name = (
        f"{asset.model_name} ({asset.serial_number})"
        if asset.model_name and asset.serial_number
        else asset.model_name or asset.serial_number or alarm_id
    )
    now_utc = datetime.now(timezone.utc)
    iso_with_z = now_utc.replace(tzinfo=timezone.utc).isoformat().replace("+00:00", "Z")

    detail_text = (
        f"{display_name} calibration status = {calibration_status}, "
        f"severity = {target_severity} at {iso_with_z}"
    )

    # Resolve asset location information
    resolved_location, system_id, system_alias = resolve_asset_location(asset)

    # Build properties base
    properties: Dict[str, Any] = {
        "Model name": asset.model_name,
        "Serial number": asset.serial_number,
        "Vendor name": asset.vendor_name,
    }

    # Only add minionId and system if we resolved them
    if system_id is not None and system_alias is not None:
        properties["minionId"] = system_id
        properties["system"] = system_alias

    # Build description based on calibration status
    verb_phrase_by_status = {
        CalibrationStatus.PAST_RECOMMENDED_DUE_DATE.value: "has exceeded",
        CalibrationStatus.APPROACHING_RECOMMENDED_DUE_DATE.value: "is approaching",
    }
    verb_phrase = verb_phrase_by_status.get(calibration_status)

    if verb_phrase:
        description = build_calibration_description(
            alarm_display_name=display_name,
            due_verb_phrase=verb_phrase,
            resolved_location=resolved_location,
            asset=asset,
        )
    else:
        description = (
            f"{display_name} {CALIBRATION_STATUS_TO_LABEL[calibration_status]}"
        )

    # Build dynamic transition properties for template resolution
    transition_properties: Dict[str, Any] = {
        "CALIBRATION_STATUS": CALIBRATION_STATUS_TO_LABEL[calibration_status],
        "ALARM_DESCRIPTION": description,
    }

    # Template strings
    display_name_template = f"{display_name} - <CALIBRATION_STATUS>"
    description_template = "<ALARM_DESCRIPTION>"

    # Create appropriate transition based on severity
    if target_severity == -1:
        transition = ClearAlarmTransition(
            occurred_at=iso_with_z,
            value=calibration_status,
            condition=calibration_status,
            detail_text=detail_text,
            properties=transition_properties,
        )
    else:
        transition = SetAlarmTransition(
            occurred_at=iso_with_z,
            severity_level=target_severity,
            value=calibration_status,
            condition=calibration_status,
            detail_text=detail_text,
            properties=transition_properties,
        )

    return CreateOrUpdateAlarmRequest(
        alarm_id=alarm_id,
        workspace=asset.workspace,
        channel=alarm_id,
        display_name=display_name_template,
        description=description_template,
        transition=transition,
        properties=properties,
    )


def should_update_alarm(existing_alarm: Alarm | None, target_severity: int) -> bool:
    """
    Checks if an alarm should be updated or not based on severity.

    Args:
        existing_alarm (Alarm): Alarm to be checked
        target_severity (int): Desired severity level (-1 to clear, 1 for low, 2 for moderate)

    Returns:
        bool: True if the alarm needs to be updated.
    """
    if existing_alarm:
        return existing_alarm.current_severity_level != target_severity
    return target_severity != -1


def upsert_alarms_for_assets(
    assets: List[Asset],
    target_severity: int,
    calibration_status: str,
    chunk_size: int = 500,
) -> List[Asset]:
    """
    Create or update alarms for a list of assets based on calibration status.

    Args:
        assets (List[Asset]): List of Asset objects to process
        target_severity (int): Desired severity level (-1 to clear, 1 for low, 2 for moderate)
        calibration_status (str): Calibration status string
        chunk_size (int): Number of assets to process per batch

    Returns:
        List[Asset]: List of assets that had their alarm state updated
    """
    assets_with_updated_state: List[Asset] = []
    if not assets:
        return assets_with_updated_state

    for i in range(0, len(assets), chunk_size):
        chunk = assets[i : i + chunk_size]

        # Map asset paths -> asset
        asset_path_to_asset: Dict[str, Asset] = {
            get_asset_path(asset): asset for asset in chunk
        }

        if not asset_path_to_asset:
            continue

        alarm_filter = " or ".join(
            f'alarmId = "{alarm_id}"' for alarm_id in asset_path_to_asset
        )

        # Query existing alarms (now automatically batches through all pages)
        query_alarm_request = QueryAlarmsWithFilterRequest(
            filter=alarm_filter,
            continuation_token=None,
            take=100,
            return_count=False,
        )

        fetch_alarms: Iterator[List[Alarm]] = __batch_query(
            query_request=query_alarm_request,
            query_func=alarm_client.query_alarms,
            paged_data_field="alarms",
        )

        alarm_map: Dict[str, Alarm] = {
            alarm.alarm_id: alarm for alarms in fetch_alarms for alarm in alarms
        }

        for alarm_id, asset in asset_path_to_asset.items():
            existing_alarm = alarm_map.get(alarm_id)

            if not should_update_alarm(existing_alarm, target_severity):
                continue
            
            action = "update" if existing_alarm else "create"
            if existing_alarm:
                print(
                    f"Updating alarm for {alarm_id} "
                    f"current_severity={existing_alarm.current_severity_level}, (target_severity={target_severity})"
                )
            else:
                print(
                    f"Creating alarm for {alarm_id} "
                    f"(target_severity={target_severity})"
                )

            request = build_transition_payload(
                asset=asset,
                alarm_id=alarm_id,
                target_severity=target_severity,
                calibration_status=calibration_status,
            )
            try:
                alarm_client.create_or_update_alarm(request)
                print(
                    f"Alarm {action} OK for {alarm_id}"
                )
                assets_with_updated_state.append(asset)
            except Exception as e:
                print(
                    f"Error {action} alarm for {alarm_id}: {e}"
                )

    return assets_with_updated_state

In [ ]:
def build_email_body(
    http_url: str,
    assets_with_past_date: List[Asset],
    assets_with_approaching_due_date: List[Asset],
) -> str:
    """
    Build email notification body with asset calibration status information.

    Generates a formatted email body containing links to assets grouped by calibration status:
    - Assets with past calibration due date
    - Assets with approaching calibration due date

    Args:
        http_url (str): Base SystemLink HTTP URL
        assets_with_past_date (List[Asset]): List of assets past their calibration due date
        assets_with_approaching_due_date (List[Asset]): List of assets approaching their calibration due date

    Returns:
        str: Formatted email body as a string with asset links
    """
    parsed = urlsplit(http_url)
    host_without_api = parsed.netloc.replace("-api", "")
    base_url = urlunsplit((parsed.scheme, host_without_api, "", "", ""))

    def asset_link(asset: Asset) -> str:
        asset_id = asset.id
        return f"{base_url}/assets/{asset_id}"

    lines: List[str] = []

    # Assets with past date
    lines.append("Assets with past calibration due date: ")
    if assets_with_past_date:
        for idx, asset in enumerate(assets_with_past_date, start=1):
            lines.append(f"{idx}. {asset_link(asset)}")
    else:
        lines.append("No new assets with past calibration due date.")

    lines.append("")  # blank line between sections

    # Assets with approaching due date
    lines.append("Assets with approaching calibration due date: ")
    if assets_with_approaching_due_date:
        for idx, asset in enumerate(assets_with_approaching_due_date, start=1):
            lines.append(f"{idx}. {asset_link(asset)}")
    else:
        lines.append("No new assets with approaching calibration due date.")

    return "\n".join(lines)

## Implementation

In [ ]:
filter_str = ""
if workspace_ids:
    conditions = " OR ".join(f'workspace = "{ws}"' for ws in workspace_ids)
    filter_str = f"({conditions})"

query_assets_request = QueryAssetsRequest(
    take=100,
    skip=0,
    filter=filter_str,
)
fetch_assets: Iterator[List[Asset]] = __batch_query(
    query_request=query_assets_request,
    query_func=asset_client.query_assets,
    paged_data_field="assets",
)
all_assets = [asset for batch in fetch_assets for asset in batch]
print(f"Total assets: {len(all_assets)}")

# Divide assets into 3 categories based on calibrationStatus
assets_with_past_date: List[Asset] = []
assets_with_approaching_due_date: List[Asset] = []
calibrated_assets: List[Asset] = []

for asset in all_assets:
    status = asset.calibration_status
    if status == CalibrationStatus.PAST_RECOMMENDED_DUE_DATE:
        assets_with_past_date.append(asset)
    elif status == CalibrationStatus.APPROACHING_RECOMMENDED_DUE_DATE:
        assets_with_approaching_due_date.append(asset)
    elif status == CalibrationStatus.OK:
        calibrated_assets.append(asset)

sb.glue("Assets with calibration past due date", len(assets_with_past_date))
sb.glue("Assets with approaching due date", len(assets_with_approaching_due_date))
sb.glue("Calibrated assets", len(calibrated_assets))

# PAST_RECOMMENDED_DUE_DATE -> severity 2
new_assets_with_past_date = upsert_alarms_for_assets(
    assets=assets_with_past_date,
    target_severity=CALIBRATION_STATUS_TO_SEVERITY[
        CalibrationStatus.PAST_RECOMMENDED_DUE_DATE.value
    ],
    calibration_status=CalibrationStatus.PAST_RECOMMENDED_DUE_DATE.value,
)

# APPROACHING_RECOMMENDED_DUE_DATE -> severity 1
new_assets_with_approaching_due_date = upsert_alarms_for_assets(
    assets=assets_with_approaching_due_date,
    target_severity=CALIBRATION_STATUS_TO_SEVERITY[
        CalibrationStatus.APPROACHING_RECOMMENDED_DUE_DATE.value
    ],
    calibration_status=CalibrationStatus.APPROACHING_RECOMMENDED_DUE_DATE.value,
)

# Send email notification if there are assets to report
if not new_assets_with_past_date and not new_assets_with_approaching_due_date:
    print(
        "No assets with calibration status changes to report - skipping email notification"
    )
else:
    mail_body = build_email_body(
        http_url,
        new_assets_with_past_date,
        new_assets_with_approaching_due_date,
    )

    send_email_notification(
        calibration_subscribers,
        "Asset due-date alerts",
        mail_body,
    )

# OK -> severity -1
upsert_alarms_for_assets(
    assets=calibrated_assets,
    target_severity=CALIBRATION_STATUS_TO_SEVERITY[CalibrationStatus.OK.value],
    calibration_status=CalibrationStatus.OK.value,
)